In [1]:
import os, getpass
env_path = '.env'
from dotenv import load_dotenv
import json
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
load_dotenv(env_path)
from datetime import datetime
from langchain_core.pydantic_v1 import constr, BaseModel, Field, validator
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage
from langchain_core.prompts.chat import ChatPromptTemplate,MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from typing_extensions import TypedDict, Annotated
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import StructuredTool
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from typing import Literal, Optional, List, Dict, Any
from langchain_core.tools import tool
import functools
import pandas as pd
import pymupdf4llm
from docx import Document
import win32com.client


llm = AzureChatOpenAI(temperature=0.7,
                        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                        openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
                        azure_deployment=os.getenv('AZURE_GPT35_MODEL')
                        )

c:\ML\CHROMA\TCSParsing\.venv\lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
c:\ML\CHROMA\TCSParsing\.venv\lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to im

In [2]:
class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,# and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    name: Optional[str] = Field(default=None, description="The name of the person")
    contact: Optional[str] = Field(default=None, description="The mobile number of the person")
    email: Optional[str] = Field(default=None, description="The Email of the person")
    dob: Optional[str] = Field(default=None, description="The date of birth of the person")
    address: Optional[str] = Field(default=None, description="The address of the person")
    job_role: Optional[str] = Field(default=None, description="The designation of the person in company")
    skills: List[str] = Field(default=None, description="The skills of the person.list of skills, programming languages, IT tools, software skills")
    years_of_experience: Optional[str] = Field(default=None, description="The years of experience of the person")
    company: Optional[str] = Field(default=None, description="The company of the person")
    education: List[str] = Field(default=None, description="The education of the person.university or high-school education qualification or degree")
    education_institute: Optional[str] = Field(default=None, description="The institute of the education")
    education_year: Optional[str] = Field(default=None, description="The year of education")
    education_degree: Optional[str] = Field(default=None, description="The degree of education")
    course_startdate: Optional[str] = Field(default=None, description="The start date of the course")
    course_enddate: Optional[str] = Field(default=None, description="The end date of the course")
    certification: List[str] = Field(default=None, description="The certification of the person")
    number_of_certifications: Optional[str] = Field(default=None, description="len(certification)")
    awards: List[str] = Field(default=None, description="The awards or achivements of the person")
    refernces: Optional[str] = Field(default=None, description="The refernces of the person")
    miscellaneous: Optional[str] = Field(default=None, description="The mislenious information of the person")
    summary: Optional[str] = Field(default=None, description="summary of person. It should not exceed more than 100 words.")

In [3]:
strctured_llm = llm.with_structured_output(schema=Person)

c:\ML\CHROMA\TCSParsing\.venv\lib\site-packages\langchain_openai\chat_models\base.py:1534: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [6]:
def pdf_to_txt_convertor(file_path_name):
    """
    Converting pdf to Markdown text.
    Text is provided to LLM for exraction
    """
    try:
        md_text = pymupdf4llm.to_markdown(file_path_name)
        return md_text
    except Exception as e:
        print(f'pdf extraction error {file_path_name}:{e}')
        return None
            

In [7]:
def docx_to_txt_convertor(file_path_name):
    """
    Converting doc to text.
    Text is provided to LLM for exraction
    """
    try:
        doc = Document(file_path_name)
        full_txt = []
        for paragraph in doc.paragraphs:
            full_txt.append(paragraph.text)
        return '\n'.join(full_txt)
    except Exception as e:
        print(f'Docx Conversion Error:{file_path_name}:{e}')
        return None

In [8]:
'''
def doc_to_txt_convertor(file_path_name):
    """
    Converting doc to text.
    Text is provided to LLM for exraction
    """
    try:
        word = win32com.client.Dispatch('word.Application')
        word.visile = False
        doc = word.Docment.open(file_path_name)
        txt = doc.Content.Text
        doc.Close()
        word.Quit()
    except Exception as e:
        print(f'Doc Conversion Error:{file_path_name}:{e}')
        return None
'''

'\ndef doc_to_txt_convertor(file_path_name):\n    """\n    Converting doc to text.\n    Text is provided to LLM for exraction\n    """\n    try:\n        word = win32com.client.Dispatch(\'word.Application\')\n        word.visile = False\n        doc = word.Docment.open(file_path_name)\n        txt = doc.Content.Text\n        doc.Close()\n        word.Quit()\n    except Exception as e:\n        print(f\'Doc Conversion Error:{file_path_name}:{e}\')\n        return None\n'

In [9]:
def extract_txt_from_resume(file_path_name):
    try:
        if file_path_name.endswith('.pdf'):
            return pdf_to_txt_convertor(file_path_name)
        elif file_path_name.endswith('.docx'):
            return docx_to_txt_convertor(file_path_name)
        else:
            print(f'unsuported file format: {file_path_name}')
            return None
    except Exception as e:
        print(f'Resume text extraction error:{e}')
        return None

In [10]:
prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are specialized agent to provide extracted information from resume."
        "If the value is not known fillvalue with null."
        "Do not make or create or generate any information which is not provided"
    ),
    (
        "human","{text}"
    )
])

In [11]:
prompt = prompt_template.invoke({"text":extract_txt_from_resume("john doe.pdf")})

In [12]:
llm_response = strctured_llm.invoke(prompt)

In [13]:
response_dict = llm_response.dict()
response_json = json.dumps(response_dict,indent = 4)

In [14]:
print(response_json)

{
    "name": "John Doe",
    "contact": "+91-9008198377",
    "email": "john.doe@hotmail.com",
    "dob": null,
    "address": "Mumbai",
    "job_role": null,
    "skills": null,
    "years_of_experience": null,
    "company": null,
    "education": null,
    "education_institute": null,
    "education_year": null,
    "education_degree": null,
    "course_startdate": null,
    "course_enddate": null,
    "certification": null,
    "number_of_certifications": null,
    "awards": null,
    "refernces": null,
    "miscellaneous": null,
    "summary": "John Doe is a highly experienced professional with 19 years of international leadership experience in project and process management. He is skilled in procurement solutions, category management, sourcing, and supplier relationship management. John holds a Master's in Information Management from Jamnalal Bajaj Institute of Management Studies, a Bachelor of Computer Application from Madurai Kamaraj University, and a Diploma in Electronics & 

In [15]:
folder_path = 'Resume Dump\SampleResumes'
folder_json_output = 'Resume Dump\output_LLMjson'
os.makedirs(folder_json_output,exist_ok=True)
for root,dirs,files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root,file)
        print('File Name: ',file)
        #print('File Path: ', file_path)
        prompt = prompt_template.invoke({"text":extract_txt_from_resume(file_path)})
        llm_response = strctured_llm.invoke(prompt)
        response_dict = llm_response.dict()
        response_json = json.dumps(response_dict,indent = 4)
        print(response_json)
        json_file = os.path.join(folder_json_output,file+'.json')
        print('File output Path: ', json_file)
        with open(json_file,'w') as file:
            json.dump(response_json,file,indent = 4)

File Name:  1.SwamyKumara 1 page.pdf
{
    "name": "KUMARA SWAMY",
    "contact": "9785645310",
    "email": "swamykumara211@gmail.com",
    "dob": "05/08/1997",
    "address": "Kolkata",
    "job_role": "Associate Software Engineer",
    "skills": [
        "Core Java",
        "Spring Boot",
        "MYSQL",
        "CSS",
        "HTML",
        "Data Structure & Algorithm"
    ],
    "years_of_experience": "1.6",
    "company": "CGI Information Systems & Management Consultants Pvt Ltd",
    "education": [
        "B. Tech in Computer Science",
        "Intermediate",
        "High School"
    ],
    "education_institute": "Haldia Institute of Technology, Haldia, India",
    "education_year": "Aug 2016 - July 2020",
    "education_degree": "B. Tech",
    "course_startdate": null,
    "course_enddate": null,
    "certification": [
        "Java Full Stack"
    ],
    "number_of_certifications": "1",
    "awards": [
        "Academic Honor Award",
        "Received appreciation from m

In [ ]:
import json

# Specify the paths of the two JSON files
file1_path = "Resume Dump/output_LLMjson/1.SwamyKumara 1 page.pdf.json"
file2_path = "Resume Dump\JsonFiles-Rchilli\1.SwamyKumara 1 page.pdf.json"

# Load the JSON data from both files
with open(file1_path, "r") as file1, open(file2_path, "r") as file2:
    json_data1 = json.load(file1)
    json_data2 = json.load(file2)

# Function to compare JSON data and calculate score
def compare_json_keys_and_values(data1, data2):
    score = 0
    identical_keys = set(data1.keys()).intersection(set(data2.keys()))
    mismatched_keys = []

    for key in identical_keys:
        if data1[key] == data2[key]:
            score += 1  # Increase score for matching values
        else:
            score -= 1  # Decrease score for mismatched values
            mismatched_keys.append((key, data1[key], data2[key]))

    return score, mismatched_keys

# Compare the JSON data
score, differences = compare_json_keys_and_values(json_data1, json_data2)

print(f"Score: {score}")
if not differences:
    print("All identical keys have matching values.")
else:
    print("The JSON files have differences in identical keys:")
    for key, value1, value2 in differences:
        print(f"Key: {key}, File1 Value: {value1}, File2 Value: {value2}")

In [ ]:
# Specify the folder path
folder_path = "Resume Dump\JsonFiles-Rchilli"

# Iterate through the files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file has a .txt extension
    if file_name.endswith('.txt'):
        # Construct the full file path
        old_file_path = os.path.join(folder_path, file_name)
        # Replace the .txt extension with .json
        new_file_name = file_name.replace('.txt', '.json')
        new_file_path = os.path.join(folder_path, new_file_name)
        # Rename the file
        os.rename(old_file_path, new_file_path)

print("All .txt files have been renamed to .json.")